In [1]:
from ipynb.fs.full.Observer import *

In [2]:
class DebugMachine(CaptureMachine):
    """ Camera Capture State Machine, primary loop:
                                 {on interaction detection} --> (interaction)            
                                 /                                      |
                __init__ ---> (idle)                                    V
                                  ^------- (observation) <--------- (debounce)
            
    """
    states = ["idle", "interaction", "debounce", "observation", "error"]
    mode = ["terrain", "add_unit", "move_unit"]
    observationThreshold = 3
    def __init__(self, captureConfiguration: CaptureConfiguration):
        self.cc = captureConfiguration
        self.mode = "unit"
        self.cc.capture()
        self.cc.setBase()
        self.cc.setReference()
        self.state = "idle"
        self.interactionDetected = False
        self.nextState = self.next_state()
        self.debounceCounter = 0
        self.cycleCounter = 0
        self.changes = {}
        self.memory = CaptureMemory()
        self.buildPrimeReference()
        self.lastChanges = None
        
    def __repr__(self):
        return f"CapMac -- {self.state}\nMode: {self.mode}\nID: {self.interactionDetected} | DC: {self.debounceCounter}"
    
    def interactionDetection(self):
        detections = {cam.camNum: cam.interactionDetection(cam.mostRecentFrame) for cam in cameras.values()}
        return sum(detections.values()) > 0

    def buildPrimeReference(self):
        self.cc.capture()
        self.primeReference = {camNum: cameras[camNum].mostRecentFrame for camNum in cameras}
    
    def updateReference(self):
        self.cc.setReference()
        
    def referenceFrameDeltas(self):
        return {camNum: cam.referenceFrameDelta() for camNum, cam in cameras.items()}
    
    def cycle(self):
        print(f"Starting Cycle {self.cycleCounter} -- S:{self.state}")
        self.cycleCounter += 1
        self.cc.capture()
        changes = self.referenceFrameDeltas()
        if sum([change is not None for change in changes.values()]):
            if changes == self.lastChanges:
                print(f"Stable changes. Committing {changes}!")
                for camNum, change in changes.items():
                    cameras[camNum].commitChange(change)
                self.cc.setReference()
            else:
                print("Unstable changes")
                self.lastChanges = changes
        else:
            print("No changes")

In [3]:
cm = DebugMachine(cc)
cm.cc.setReference()

Saw interaction? False


In [18]:
if __name__ == "__main__":
    cm.cycle()

Starting Cycle 13 -- S:idle
Stable changes. Committing {0: [606.0, 831.0] by 66.0,70.0, 2: [587.0, 1240.0] by 63.0,75.0, 4: [726.0, 805.0] by 71.0,75.0}!
Deleting [605.0, 831.0] by 66.0,69.0
Deleting [588.0, 1242.0] by 66.0,76.0
Deleting [726.0, 807.0] by 80.0,74.0


In [19]:
cameras[0].changes

[]